In [4]:
import pandas as pd
import os
import datetime

In [5]:
# Graph Templates
def create_pd3_template(lld_templates: str, gpm_templates: str, lld_edges: str, lld_gpm_edges: str) -> str:
    PD3_Templeate = f"""
classDiagram
    direction LR

    %% LLD Actions\n{lld_templates}
    %% GPM Actions\n{gpm_templates}
    %% LLD Edges\n{lld_edges}
    %% LLD-GPM Edges\n{lld_gpm_edges}
    """
    return PD3_Templeate

def create_GPM_templates(GPM_IDs: list[int], GPM_inputs: list[str], GPM_names: list[str], GPM_outputs: list[str]) -> str:
    GPMs = ""
    for row in range(len(GPM_names)):
        GPMs += f"""
        class GPM_{GPM_IDs[row]}["GPM_{GPM_IDs[row]}: {GPM_names[row]}"] {{
            Input: {GPM_inputs[row]}
            Output: {GPM_outputs[row]}
        }}
        \n"""
    return GPMs

def create_LLD_templates(LLD_IDs: list[int], LLD_inputs: list[str], LLD_names: list[str], LLD_outputs: list[str]) -> str:
    LLDs = ""
    for row in range(len(LLD_names)):
        LLDs += f"""
        class LLD_{LLD_IDs[row]}["LLD_{LLD_IDs[row]}: {LLD_names[row]}"] {{
            Input: {LLD_inputs[row]}
            Output: {LLD_outputs[row]}
        }}
        \n"""
    return LLDs

def create_LLD_edges(LLD_IDs: list[int]) -> str:
    LLD_edges = ""
    for row in range(1, len(LLD_IDs)):
        LLD_edges += f"""LLD_{LLD_IDs[row-1]} --> LLD_{LLD_IDs[row]}\n"""
    return LLD_edges

def create_LLD_GPM_edges(LLD_IDs: list[int], LLD_GPM_edges: list[int]) -> str:
    generalization_edges = ""
    for row in range(len(LLD_GPM_edges)):
        generalization_edges += f"""LLD_{LLD_IDs[row]} <|.. GPM_{LLD_GPM_edges[row]} : Implements\n"""
    return generalization_edges


In [6]:
# PartOf Relationship Template
def create_GPM_partof_diagram(gpm_templates: str, gpm_edges: str) -> str:
    PD3_Templeate = f"""
classDiagram
    direction BT

    %% GPM Actions\n{gpm_templates}
    %% GPM Edges\n{gpm_edges}
    """
    return PD3_Templeate

def create_GPM_edges(GPM_IDs: list[str], gpm_partOfs: list[str]) -> str:
    edges_str = ""
    for row in range(len(gpm_partOfs)):
        edges_str += f"GPM_{GPM_IDs[row]} *-- GPM_{gpm_partOfs[row]}\n"
    return edges_str

In [ ]:
# Which LLD actions GPM actions derive from
def create_LLD_templates_with_Log(LLD_logs: list[int], LLD_IDs: list[int], LLD_inputs: list[str], LLD_names: list[str], LLD_outputs: list[str]) -> str:
    LLDs = ""
    for row in range(len(LLD_names)):
        LLDs += f"""
        class LLD_{LLD_IDs[row]}["LLD_{LLD_IDs[row]}: {LLD_names[row]}"] {{
            Log: {LLD_logs[row]}
            Input: {LLD_inputs[row]}
            Output: {LLD_outputs[row]}
        }}
        \n"""
    return LLDs

def create_GPM_reference_template(lld_templates: str, gpm_templates: str, lld_gpm_edges: str) -> str:
    PD3_Templeate = f"""
classDiagram
    direction LR
    %% GPM Actions\n{gpm_templates}
    %% LLD Actions\n{lld_templates}
    %% LLD-GPM Edges\n{lld_gpm_edges}
    """
    return PD3_Templeate

def GPM_LLD_references(df_pd3, df_GPM, LLD_Logs = "Log", LLD_IDs = "Action ID", LLD_Inputs="Input", LLD_Names="Action", LLD_Outputs="Output", GPM_IDs="ClassID", GPM_Inputs="ClassInput", GPM_Names="ClassName", GPM_Outputs="ClassOutput", LLD_GPM_Edges="ClassID"):
    """
    Args: 
        Information on the correspondence of GPM actions to LLD actions.
    Returns:
        A template of each GPM action with its LLD action references.
    """
    LLD_templates = create_LLD_templates_with_Log(df_pd3[LLD_Logs].tolist(), df_pd3[LLD_IDs].tolist(), df_pd3[LLD_Inputs].tolist(), df_pd3[LLD_Names].tolist(), df_pd3[LLD_Outputs].tolist())
    GPM_templates = create_GPM_templates(df_GPM[GPM_IDs].tolist(), df_GPM[GPM_Inputs].tolist(), df_GPM[GPM_Names].tolist(), df_GPM[GPM_Outputs].tolist())
    LLD_GPM_edges = create_LLD_GPM_edges(df_pd3[LLD_IDs].tolist(), df_pd3[LLD_GPM_Edges].tolist())
    gpm_lld_references = create_GPM_reference_template(LLD_templates, GPM_templates, LLD_GPM_edges)
    return gpm_lld_references

In [8]:
def files_in_dir(directory_path):
    files_and_dirs = os.listdir(directory_path)
    print(files_and_dirs)

def get_current_datetime_components():
    """Returns the current year, month, day, hour, and minute as formatted strings."""
    now = datetime.datetime.now()
    return f"{now.strftime("%Y")}-{now.strftime("%m")}-{now.strftime("%d")}-{now.strftime("%H")}-{now.strftime("%M")}"

def create_mermaid_diagram(df_pd3, df_GPM, LLD_IDs = "Action ID", LLD_Inputs="Input", LLD_Names="Action", LLD_Outputs="Output", GPM_IDs="ClassID", GPM_Inputs="ClassInput", GPM_Names="ClassName", GPM_Outputs="ClassOutput", LLD_GPM_Edges="ClassID"):
    """
    Args:
        df (pandas.DataFrame): DataFrame containing the class hierarchy data.
    Returns:
        str: Mermaid diagram code with Input, Class name and Output, representing the class hierarchy.
    """
    LLD_actions = create_LLD_templates(df_pd3[LLD_IDs].tolist(), df_pd3[LLD_Inputs].tolist(), df_pd3[LLD_Names].tolist(), df_pd3[LLD_Outputs].tolist())
    GPM_actions = create_GPM_templates(df_GPM[GPM_IDs].tolist(), df_GPM[GPM_Inputs].tolist(), df_GPM[GPM_Names].tolist(), df_GPM[GPM_Outputs].tolist())
    LLD_edges = create_LLD_edges(df_pd3[LLD_IDs].tolist())
    LLD_GPM_edges = create_LLD_GPM_edges(df_pd3[LLD_IDs].tolist(), df_pd3[LLD_GPM_Edges].tolist())
    pd3_template = create_pd3_template(LLD_actions, GPM_actions, LLD_edges, LLD_GPM_edges)
    return pd3_template

def create_GPM_containers(df_GPM, GPM_IDs="ClassID", GPM_Inputs="ClassInput", GPM_Names="ClassName", GPM_Outputs="ClassOutput", GPM_Parents="PartOf"):
    """
    Args:
        df (pandas.DataFrame): DataFrame containing the class hierarchy data.
    Returns:
        str: Mermaid diagram code with Input, Class name and Output, representing the class hierarchy.
    """
    GPM_actions = create_GPM_templates(df_GPM[GPM_IDs].tolist(), df_GPM[GPM_Inputs].tolist(), df_GPM[GPM_Names].tolist(), df_GPM[GPM_Outputs].tolist())
    GPM_edges = create_GPM_edges(df_GPM[GPM_IDs].tolist(), df_GPM[GPM_Parents].tolist())
    GPM_containers = create_GPM_partof_diagram(GPM_actions, GPM_edges)
    return GPM_containers

In [5]:
files_in_dir("./target/")

['ma_welding_GPM_classes-2025-12-22-13-29.xlsx', 'ma_welding_LLDs_with_GPM-2025-12-22-13-29.xlsx', 'merged_lld_gpm-2025-12-22-13-57.xlsx']


In [9]:
df_GPM = pd.read_excel("./target/ma_welding_GPM_classes-2025-12-22-13-29.xlsx")
df_pd3 = pd.read_excel("./target/merged_lld_gpm-2025-12-22-13-57.xlsx")
print(df_GPM.head(1))
df_each_log = df_pd3.groupby("Log", as_index=False)
df_each_log.head(1)

   ClassID   ClassInput     ClassName  \
0        1  生産ラインの不具合一覧  解決対象不具合を選定する   

                                         ClassOutput  PartOf  \
0  解決対象不具合（例：300Dフロントピラー精度不良, 300Dトルーフの隙間, 120D/S...       0   

                                 RelationKnowledge  
0  起点タスク。Tips: 前後関係の最初に来る選定工程であり上位に親はないためPartOf=0。  


,Log,Action ID,Input,Action,Output,Intention,Rationale,Annotation,Tools,Engineering knowledge,ClassID,ClassName_x,Knowledge,Timestamp,ClassInput,ClassName_y,ClassOutput,PartOf,RelationKnowledge
0,1,1,治具に固定された300Dフロントピラー,300dフロントピラーの三次元形状を測定する,300Dフロントピラーの三次元形状データ；300DフロントピラーのCADデータ,組み付け精度を確認するために、部品に不具合がないかを知りたい,no rationale,no annotation,三次元形状測定機,no engineering knowledge,1,解決対象不具合を選定する,キーワード:300Dフロントピラー/治具; 動詞=測定; 目的=組付け精度確認,2024-01-01 00:00:00,生産ラインの不具合一覧,解決対象不具合を選定する,"解決対象不具合（例：300Dフロントピラー精度不良, 300Dトルーフの隙間, 120D/S...",0,起点タスク。Tips: 前後関係の最初に来る選定工程であり上位に親はないためPartOf=0。
36,2,37,治具にセット済みの300Dトルーフ,目視や手触りによる外観チェックで300Dトルーフの不具合を観察する,300Dトルーフの１打点目がパネルズレがあり、\n打点すると位置ズレが起こる,どんな不具合があるのかを知りたい,no rationale,no annotation,Tool\n目視、手触り,知識：\n・外観チェックで隙間や位置ズレなどの目立ている不具合の確認ができる\n・隙間は1m...,7,打点位置の妥当性を確認する,動詞=外観点検; 対象=300Dトルーフ,2024-01-01 00:36:00,"組付け体, 測定基準, 対象ライン（トリムライン等）",隙間（パネルズレ）を測定する,隙間データ,2,隙間測定は現状把握の一部。Domain: トリムライン周りの隙間重視。
72,3,73,セット済みの120D の部品,部品の三次元形状を測定する,120D の三次元形状データ,部品の形状に異常ないかを知りたい,no rationale,no annotation,三次元測定機,no engineering knowledge,1,NaN,動詞=測定; 対象=120D,2024-01-01 01:12:00,生産ラインの不具合一覧,解決対象不具合を選定する,"解決対象不具合（例：300Dフロントピラー精度不良, 300Dトルーフの隙間, 120D/S...",0,起点タスク。Tips: 前後関係の最初に来る選定工程であり上位に親はないためPartOf=0。
100,4,101,no input,単品の32Dセンターフロアの精度を測定する,測定結果：\n１：単品の①②③面精度が悪い\n２：基準Ⅰ、Ⅱの穴径精度が悪い,部品に不具合がないかを知りたい,no rationale,no annotation,no tools,Knowledge：\n知識：\n・③の面位置が公差土1.5に対して・1.6～-1.8と公差...,1,NaN,動詞=測定; 32Dセンターフロア単品精度,2024-01-01 01:40:00,生産ラインの不具合一覧,解決対象不具合を選定する,"解決対象不具合（例：300Dフロントピラー精度不良, 300Dトルーフの隙間, 120D/S...",0,起点タスク。Tips: 前後関係の最初に来る選定工程であり上位に親はないためPartOf=0。
120,5,121,no input,部品460B ラダーAssyに不具合がないかを調べる,ロッカーインナRr×クロスNo1\n\nW方向外開き、縦面隙あり,部品に不具合がないかを知りたい,no rationale,no annotation,Tool：\n目視,no engineering knowledge,7,NaN,動詞=外観点検; ラダーAssy,2024-01-01 02:00:00,"組付け体, 測定基準, 対象ライン（トリムライン等）",隙間（パネルズレ）を測定する,隙間データ,2,隙間測定は現状把握の一部。Domain: トリムライン周りの隙間重視。


In [7]:
for group in df_each_log.groups.keys():
    graph = create_mermaid_diagram(df_each_log.get_group(group), df_GPM)
    with open(f"./outputs/graph_{group}-{get_current_datetime_components()}.mmd", "w") as f:
        f.write(graph)
    print(f"Log: {group} => The PD3 has been created.")

Log: 1 => The PD3 has been created.
Log: 2 => The PD3 has been created.
Log: 3 => The PD3 has been created.
Log: 4 => The PD3 has been created.
Log: 5 => The PD3 has been created.


In [8]:
containers = create_GPM_containers(df_GPM)
with open(f"./outputs/GPM_containers-{get_current_datetime_components()}.mmd", "w") as f:
    f.write(containers)

In [15]:
gpm_lld_references_diagram = GPM_LLD_references(df_pd3, df_GPM)
with open(f"./outputs/gpm_references-{get_current_datetime_components()}.mmd", "w") as f:
    f.write(gpm_lld_references_diagram)